In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

# --- NeuroLoft Aesthetic Config ---
plt.style.use('dark_background')
sns.set_palette("viridis")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"🟢 System Ready. Compute Node: {device}")
if device.type == 'cuda':
    print(f"🚀 GPU: {torch.cuda.get_device_name(0)}")

## 2. 💾 Data Plane Activation
*Ingesting raw pixel data and reshaping for the visual cortex.*

In [ ]:
# Load Data
# Note: On Kaggle, paths will be '/kaggle/input/digit-recognizer/train.csv'
try:
    train_df = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
    test_df = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')
    print("✅ Data Loaded Successfully from Kaggle Input")
except FileNotFoundError:
    print("⚠️ Local mode: Generating dummy data for testing structure")
    # Fix: Labels must be 0-9 for MNIST, pixels 0-255
    train_df = pd.DataFrame(np.random.randint(0, 255, (100, 785)), columns=['label'] + [f'pixel{i}' for i in range(784)])
    train_df['label'] = np.random.randint(0, 10, 100) # Force labels to be 0-9
    test_df = pd.DataFrame(np.random.randint(0, 255, (50, 784)), columns=[f'pixel{i}' for i in range(784)])

# Preprocessing
X = train_df.drop('label', axis=1).values / 255.0
y = train_df['label'].values
X_test = test_df.values / 255.0

# Reshape for CNN (Batch, Channel, Height, Width)
X = X.reshape(-1, 1, 28, 28)
X_test = X_test.reshape(-1, 1, 28, 28)

# Split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

# To Tensors
train_tensor = TensorDataset(torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.long))
val_tensor = TensorDataset(torch.tensor(X_val, dtype=torch.float32), torch.tensor(y_val, dtype=torch.long))
test_tensor = torch.tensor(X_test, dtype=torch.float32)

loaders = {
    'train': DataLoader(train_tensor, batch_size=64, shuffle=True),
    'val': DataLoader(val_tensor, batch_size=64, shuffle=False)
}

print(f"📊 Training Tensor Shape: {X_train.shape}")

In [ ]:
# Visualize
plt.figure(figsize=(10, 4))
for i in range(5):
    plt.subplot(1, 5, i+1)
    plt.imshow(X_train[i][0], cmap='magma')
    plt.title(f"Label: {y_train[i]}", color='#40E0D0')
    plt.axis('off')
plt.suptitle("👁️ Visual Input Stream", color='white', fontsize=14)
plt.show()

## 3. 🧠 Intelligence Core (CNN)
*Constructing the Convolutional Neural Network architecture.*

In [ ]:
class NeuroCNN(nn.Module):
    def __init__(self):
        super(NeuroCNN, self).__init__()
        self.conv_block1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.conv_block2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.flatten = nn.Flatten()
        self.fc = nn.Sequential(
            nn.Linear(64 * 7 * 7, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 10)
        )
        
    def forward(self, x):
        x = self.conv_block1(x)
        x = self.conv_block2(x)
        x = self.flatten(x)
        x = self.fc(x)
        return x

model = NeuroCNN().to(device)
print(model)

## 4. 🔮 Neural Interface Simulation (Training)
*Optimizing synaptic weights.*

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

epochs = 5 # Quick run for demo
history = {'loss': [], 'acc': []}

print("🚀 Initiating Training Sequence...")

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for images, labels in loaders['train']:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
    epoch_acc = 100 * correct / total
    history['loss'].append(running_loss / len(loaders['train']))
    history['acc'].append(epoch_acc)
    print(f"Epoch [{epoch+1}/{epochs}] | Loss: {running_loss/len(loaders['train']):.4f} | Acc: {epoch_acc:.2f}%")

print("✅ Training Complete.")

In [ ]:
# --- Submission ---
model.eval()
with torch.no_grad():
    test_preds = model(test_tensor.to(device))
    _, predicted_labels = torch.max(test_preds, 1)

submission = pd.DataFrame({
    'ImageId': range(1, len(predicted_labels) + 1),
    'Label': predicted_labels.cpu().numpy()
})

submission.to_csv('submission.csv', index=False)
print("💾 Submission file 'submission.csv' generated successfully.")
submission.head()